Parte 4: Insights y Recomendaciones 


In [0]:
%sql
-- ¿Cuál fue el mes con mejor desempeño en ventas?
SELECT MONTH, SUM(TOTAL_SALES_TAX) as venta_total
FROM workspace.operations.gold_sales_daily_summary
GROUP BY 1 ORDER BY 2 DESC LIMIT 1;

El mes de Julio fue el que tuvo mejor desempeño en ventas. 
Esto puede ser debido al inicio de vacaciones escolares.

In [0]:
%sql
--¿Cuál es el margen bruto promedio de la compañía?
SELECT AVG(PERCENTAGE_GROSS_MARGIN) as margen_promedio_cia
FROM workspace.operations.gold_sales_daily_summary;

Farmatodo tiene un margen bruto promedio del 45%, esto indica una estructura de costos saludable.

In [0]:
%sql
--¿Que día de la semana genera más ventas?
SELECT DAY_OF_WEEK, SUM(TOTAL_SALES_TAX) as venta_total
FROM workspace.operations.gold_sales_daily_summary
GROUP BY 1 ORDER BY 2 DESC LIMIT 1;

Los días martes son los días que se genera más ventas. Es bueno saber está información para optimizar los turnos de personal en tienda y la logística de despacho.

Análisis por Producto

In [0]:
%sql
--¿Cuál es el departamento con mayor margen bruto?
SELECT DEPT_NAME, SUM(GROSS_MARGIN) as margen_total
FROM workspace.operations.gold_sales_daily_summary
GROUP BY 1 ORDER BY 2 DESC LIMIT 1;

El departamento con mayor margen bruto es el de nutrición. Es decir, es el departamento que mayor ganancia genera después de cubrir los costos de los productos.

In [0]:
%sql
--¿Qué marca genera más ventas?
SELECT BRAND_NAME, SUM(TOTAL_SALES_TAX) as venta_total
FROM workspace.operations.gold_sales_daily_summary
GROUP BY 1 ORDER BY 2 DESC LIMIT 1;

La marca que genera más ventas es la génerica. Esto puede ser un indicador que el factor que más influye en la decisión de compra del cliente es cuánto cuesta el producto.

In [0]:
%sql
--¿Cuáles son los 3 productos con menor rotación (unidades vendidas)?
SELECT PRODUCT_DESCRIPTION, SUM(UNITS_SOLD) as unidades
FROM workspace.operations.gold_sales_daily_summary
GROUP BY 1 ORDER BY 2 ASC LIMIT 3;

Los tres productos con menos rotación son "Aun Cremas Generico", "Política Vitaminas Generico" y "A Antibioticos Advanced". Estos productos pueden estar ocupando espacio sin generar retorno, se podrían hacer promociones para venderlos.

Análisis Geográfico

In [0]:
%sql
--¿Qué región genera más ventas?¿Y cual tiene mejor margen?
SELECT REGION_NAME, SUM(TOTAL_SALES_TAX) AS venta_total, AVG(PERCENTAGE_GROSS_MARGIN) AS margen_promedio
FROM workspace.operations.gold_sales_daily_summary
GROUP BY 1 ORDER BY 2 DESC;
    



La región Centro es el principal generador de ingresos para Farmatodo, mientras que la región Capital tiene mejor margen. Sin embargo, todas las regiones tienen margenes muy parecidos.

In [0]:
%sql
--¿Cuáles son las 5 tiendas con mejor desempeño?
SELECT STORE_NAME, CITY, REGION_NAME, SUM(TOTAL_SALES_TAX) AS venta_total FROM workspace.operations.gold_sales_daily_summary GROUP BY 1,2,3 ORDER BY 4 DESC LIMIT 5;

De las 5 tiendas con mejor desempeño hay dos en Maturin, una en Caracas, una en Maracaibo y una en Barcelona. Es notable que las dos mejores tiendas son en Maturin, puede sugerir una ubicación estrátegica de estas tiendas.

In [0]:
%sql
--¿Existe diferencia significativa de ventas entre formatos de tienda?
SELECT FORMAT_NAME, COUNT(DISTINCT STORE_ID) AS cantidad_tiendas, SUM(TOTAL_SALES_TAX) AS venta_total, SUM(TOTAL_SALES_TAX)/ COUNT(DISTINCT STORE_ID) AS venta_promedio
 FROM workspace.operations.gold_sales_daily_summary GROUP BY 1 ORDER BY 3 DESC;

No existe una diferencia significativa en ventas entre formatos cuando se analiza el desempeño promedio por tienda.

Análisis de Canales

In [0]:
%sql
--¿Qué porcentaje de ventas proviene de canales digitales (E-Commerce + App)?
SELECT 
SUM(CASE WHEN CHANNEL_NAME in ('E-Commerce','App Movil') THEN TOTAL_SALES_TAX ELSE 0 END)*100/
SUM(TOTAL_SALES_TAX) AS porcentaje_digital
FROM workspace.operations.gold_sales_daily_summary

Un gran porcentaje de ventas viene de los canales digitales, con un 49.9% de la ventas proveniendo de ellos. Esto significa que se debería seguir impulsando la venta a través de estos canales.

In [0]:
%sql
--¿Cuál es el ticket promedio por canal?
SELECT 
    CHANNEL_NAME, 
    AVG(AVERAGE_TICKET) AS ticket_promedio_canal
FROM workspace.operations.gold_sales_daily_summary
GROUP BY 1 
ORDER BY 2 DESC;

El ticket promedio es muy similar en todos los canales, lo que indica que el comportamiento de gasto por transacción es consistente entre tienda física y canales digitales.

In [0]:
%sql
--¿Qué canal tiene mejor margen bruto?
SELECT CHANNEL_NAME, 
    SUM(GROSS_MARGIN) AS margen_total,
    AVG(PERCENTAGE_GROSS_MARGIN) AS porcentaje_margen_promedio
FROM workspace.operations.gold_sales_daily_summary
GROUP BY 1 
ORDER BY 3 DESC
LIMIT 1;

El E-Commerce es el canal de venta con el mejor margen bruto, esto sugiere que puede tener una estructura de costos comerciales más eficiente que los canales físicos.

Análisis de Proveedores

In [0]:
%sql
--¿Cuáles son los 3 proveedores más importantes por volumen de venta?
SELECT 
    SUPPLIER_NAME, 
    SUM(TOTAL_SALES_TAX) AS venta_total
FROM workspace.operations.gold_sales_daily_summary
GROUP BY 1 
ORDER BY 2 DESC 
LIMIT 3;

Los tres proveedores más importantes son Pfizer Venezuela, Johnson & Johnson y Sanofi Venezuela. Esto señala que la medicina y el cuidado personal con marcas reconocidas sigue siendo bastante importante para los clientes de Farmatodo.

In [0]:
%sql
--¿Qué porcentaje de ventas totales representan los top 3 proveedores?
WITH top_proveedores AS (
    SELECT SUM(TOTAL_SALES_TAX) AS venta_top_3
    FROM (
        SELECT SUM(TOTAL_SALES_TAX) AS TOTAL_SALES_TAX
        FROM workspace.operations.gold_sales_daily_summary
        GROUP BY SUPPLIER_NAME
        ORDER BY 1 DESC
        LIMIT 3
    )
)
SELECT 
    (venta_top_3 * 100.0 / (SELECT SUM(TOTAL_SALES_TAX) FROM workspace.operations.gold_sales_daily_summary)) AS porcentaje_participacion_top3
FROM top_proveedores;

Los top 3 proveedores representan un porcentaje de ventas totales bastante significativo con un 31.5%. Se puede presenciar la importancia de estos proveedores para Farmatodo.

Análisis Monetario

In [0]:
%sql
--¿Qué porcentaje de transacciones se realizan en dólares vs bolívares?
SELECT 
    CURRENCY_ABR, 
    SUM(TOTAL_TRANSACTIONS) AS total_transacciones,
    (SUM(TOTAL_TRANSACTIONS) * 100.0 / SUM(SUM(TOTAL_TRANSACTIONS)) OVER()) AS porcentaje_uso
FROM workspace.operations.gold_sales_daily_summary
GROUP BY 1;

El porcentaje de transacciones en bolívares es del 70.1% mientras que en dólares es 29.9%. Se puede apreciar que la gran mayoría de clientes de Farmatodo paga en bolívares. Esto puede ser debido a la brecha cambiaria en el 2025.

In [0]:
%sql
--¿Hay diferencia en el ticket promedio según la moneda?
SELECT 
    CURRENCY_ABR, 
    AVG(AVERAGE_TICKET) AS ticket_promedio_moneda
FROM workspace.operations.gold_sales_daily_summary
GROUP BY 1;

El ticket promedio es similar con las dos monedas, lo que indica que el comportamiento de gasto por transacción es consistente a pesar de la moneda utilizada.

Parte 4.2 Documento de Insights 

Resumen Ejecutivo

1. Casi la mitad de las ventas (49.9%) provienen de canales digitales (App Movil y E-Commerce), consolidando la estrategia de apostar por la tecnología que tiene Farmatodo.
2. Farmatodo mantiene un margen bruto promedio saludable del 45%, con el canal de E-Commerce liderando la rentabilidad.
3. El ticket promedio es notablemente consistente entre canales y monedas, lo que indica un comportamiento de gasto uniforme en la base de clientes.
4. El negocio muestra una fuerte dependencia de los proveedores clave (Top 3 concentran el 31.5% de la venta), con un pico de actividad los días martes y durante el mes de julio.

Hallazgos Detallados
1. El departamento de Nutrición es el motor de rentabilidad, mientras que la marca Genérica es la preferida por volumen. 
2. La región Centro es la potencia en volumen de ventas, pero la región Capital logra optimizar mejor sus márgenes por poca cantidad. No se observan diferencias críticas entre formatos de tienda. 
3. El 70.1% de las transacciones se mantienen en bolívares, posiblemente influenciado por la brecha cambiaria del año 2025.

Oportunidades Identificadas
1. Los productos "Aun Cremas Generico", "Política Vitaminas Generico" y "A Antibioticos Advanced" presentan rotación nula o mínima, representando capital inmovilizado. 
2. Existe una oportunidad de captar mayor flujo de dólares (actualmente en 29.9%) mediante incentivos como descuentos/promociones, dado que el ticket promedio en dólares ya es competitivo.
3. Se puede impulsar aún más el crecimiento de los canales digitales (actualmente en 49.9%) mediante implementación de programas de lealtad en la aplicaión. Por ejemplo, después de tantas compras delivery gratis o iniciativas parecidas.

Recomendaciones Accionables 
1. Recomendación: Plan de liquidación de stock de baja rotación. Insight: 3 productos identificados con rotación crítica al final del trimestre. Impacto esperado: Alto (Liberación de flujo de caja y espacio en anaquel).Acción: El equipo de compras puede ejecutar promociones de "2x1" o descuentos del 30% en estos productos específicos para liquidar stock.
2. Recomendación: Refuerzo de operaciones los días martes. Insight: Los martes registran el mayor volumen de ventas semanal.  Impacto esperado: Medio (Mejora en la experiencia del cliente y reducción de tiempos de espera).  Acción: Gerencia de Tiendas puede ajustar los turnos de personal y asegurar que el reabastecimiento de anaqueles se complete el lunes por la noche.
3. Recomendación: Programa de incentivo "Pago en dólares". Insight: El 70.1% de las transacciones son en VES, pero el ticket promedio es igual en ambas monedas. Impacto esperado: Medio (Protección ante volatilidad cambiaria).  Acción: Mercadeo puede lanzar descuentos exclusivos para compras pagadas en USD.
4. Recomendación: Impulso del margen en E-Commerce. Insight: El E-Commerce es el canal con mejor porcentaje de margen bruto.  Impacto esperado: Bajo (Los canales tienen un margen bastante parecido). Acción: El equipo que trabaja con el E-Commerce puede priorizar en el algoritmo de búsqueda de la web/app los productos del departamento de Nutrición, que es el de mayor margen.
5. Recomendación: Negociación con los Top 3 proveedores. Insight: Pfizer, Johnson & Johnson y Sanofi representan el 31.5% de la facturación total. Impacto esperado: Alto (Mejora de términos comerciales y descuentos por volumen).  Acción: El Director de Compras puede establecer negociaciones para asegurar exclusividades o mejores plazos de pago dado el volumen de compra.